### Project: Analyze fluctuations in HTTP request latency data to determine reliability

Appendix:
- Prepare data:
	- Create sample data from script

- Analyze data
	- Load data to dataframe
	

- Calculate SLI
	- Bin data to bucket latency (le)
	- Explore data 
	- Calculate relative change (pct_change) on rolling windows
		Formula: xxx
	- Calculate percentile: [50th, 90th, 95th, 99th]
	
- Apply model